## 펑션 콜링 기초

현재시간을 구하는 것을 해보면서 펑션 콜링에 대해서 알아보자

In [1]:
from stock_functions import get_yf_stock_info, get_yf_stock_history, get_yf_stock_recommendations, tools 
from openai import OpenAI
from dotenv import load_dotenv
import os
import json

In [2]:
load_dotenv(dotenv_path="../.env")
api_key = os.getenv("OPENAI_API_KEY")  # 환경 변수에서 API 키 가져오기

client = OpenAI(api_key=api_key)  # 오픈AI 클라이언트의 인스턴스 생성

In [3]:
def get_ai_response(messages, tools=None):
    response = client.chat.completions.create(
        model="gpt-4o",  # 응답 생성에 사용할 모델 지정
        messages=messages,  # 대화 기록을 입력으로 전달
        tools=tools,  # 사용 가능한 도구 목록 전달
    )
    return response  # 생성된 응답 내용 반환

In [4]:
messages = [
    {"role": "system", "content": "너는 사용자를 도와주는 상담사야."},
]

user_input = "Microsoft 주식 정보는?"

messages.append({"role": "user", "content": user_input})

ai_response = get_ai_response(messages, tools=None)
ai_message = ai_response.choices[0].message
print(ai_message)

print("AI\t: " + ai_message.content)

ChatCompletionMessage(content='죄송하지만 실시간 주식 정보는 제공할 수 없습니다. 하지만 Microsoft 주식에 관한 과거의 정보나 일반적인 조언을 필요로 한다면 도와드릴 수 있습니다. 주식의 가격이나 최신 정보는 금융 뉴스 웹사이트나 주식 거래 플랫폼을 통해 확인하실 수 있습니다.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None)
AI	: 죄송하지만 실시간 주식 정보는 제공할 수 없습니다. 하지만 Microsoft 주식에 관한 과거의 정보나 일반적인 조언을 필요로 한다면 도와드릴 수 있습니다. 주식의 가격이나 최신 정보는 금융 뉴스 웹사이트나 주식 거래 플랫폼을 통해 확인하실 수 있습니다.


### 펑션 콜링 해보기

In [5]:
from stock_functions import get_yf_stock_info, get_yf_stock_history, get_yf_stock_recommendations, tools 

messages = [
    {"role": "system", "content": "너는 사용자를 도와주는 상담사야."},
]

user_input = "Microsoft 주식 정보는?"

messages.append({"role": "user", "content": user_input})

ai_response = get_ai_response(messages, tools=tools)
ai_message = ai_response.choices[0].message
print(ai_message)

print("AI\t: " + ai_message.content)

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_W38Ld4lHiXCbPoGXcOVABdSV', function=Function(arguments='{"ticker": "MSFT"}', name='get_yf_stock_info'), type='function'), ChatCompletionMessageFunctionToolCall(id='call_0LBDHrq6KaCSVG8OaHJQxUXx', function=Function(arguments='{"ticker": "MSFT", "period": "1d"}', name='get_yf_stock_history'), type='function'), ChatCompletionMessageFunctionToolCall(id='call_gNdtGVpIlgilLKXyo8vq1j4T', function=Function(arguments='{"ticker": "MSFT"}', name='get_yf_stock_recommendations'), type='function')])


TypeError: can only concatenate str (not "NoneType") to str

In [8]:
from stock_functions import get_yf_stock_info, get_yf_stock_history, get_yf_stock_recommendations, tools

messages = [
    {"role": "system", "content": "너는 사용자를 도와주는 상담사야."},
]

user_input = "Microsoft 주식 정보는?"

messages.append({"role": "user", "content": user_input})

ai_response = get_ai_response(messages, tools=tools)
ai_message = ai_response.choices[0].message
print(ai_message) # GPT 응답 확인용

tool_calls = ai_message.tool_calls  # AI 응답에 포함된 tool_calls를 가져옵니다.
if tool_calls:  # tool_calls가 있는 경우
    for tool_call in tool_calls:
        tool_name = tool_call.function.name # 실행해야한다고 판단한 함수명 받기
        tool_call_id = tool_call.id         # tool_call 아이디 받기    
        arguments = json.loads(tool_call.function.arguments) # (1) 문자열을 딕셔너리로 변환    
        
        if tool_name == "get_yf_stock_info":  # 만약 tool_name이 "get_yf_stock_info"이라면
            messages.append({
                "role": "function",  # role을 "function"으로 설정
                "tool_call_id": tool_call_id,
                "name": tool_name,
                "content": get_yf_stock_info(arguments.get('ticker')),
            })
        elif tool_name == "get_yf_stock_history":  # 만약 tool_name이 "get_yf_stock_history"이라면
            messages.append({
                "role": "function",  # role을 "function"으로 설정
                "tool_call_id": tool_call_id,
                "name": tool_name,
                "content": get_yf_stock_history(arguments.get('ticker'), arguments.get('period')),
            })
        elif tool_name == "get_yf_stock_recommendations":  # 만약 tool_name이 "get_yf_stock_recommendations"이라면
            messages.append({
                "role": "function",  # role을 "function"으로 설정
                "tool_call_id": tool_call_id,
                "name": tool_name,
                "content": get_yf_stock_recommendations(arguments.get('ticker')),
            })
    messages.append({"role": "system", "content": "이제 주어진 결과를 바탕으로 답변할 차례다."})  # 함수 실행 완료 메시지 추가
    ai_response = get_ai_response(messages, tools=tools) # 다시 GPT 응답 받기
    ai_message = ai_response.choices[0].message

messages.append(ai_message)  # AI 응답을 대화 기록에 추가하기

print("AI\t: " + ai_message.content)

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_axqT6f8JUCBrWdQRh5wHUbyx', function=Function(arguments='{"ticker": "MSFT"}', name='get_yf_stock_info'), type='function'), ChatCompletionMessageFunctionToolCall(id='call_4BnGS6orn2VDnz9HpH9xvPaD', function=Function(arguments='{"ticker": "MSFT", "period": "1mo"}', name='get_yf_stock_history'), type='function'), ChatCompletionMessageFunctionToolCall(id='call_sgvhBUUMlB0RLEfi15MocKqM', function=Function(arguments='{"ticker": "MSFT"}', name='get_yf_stock_recommendations'), type='function')])
{'address1': 'One Microsoft Way', 'city': 'Redmond', 'state': 'WA', 'zip': '98052-6399', 'country': 'United States', 'phone': '425 882 8080', 'website': 'https://www.microsoft.com', 'industry': 'Software - Infrastructure', 'industryKey': 'software-infrastructure', 'industryDisp': 'Software - Infrastructure', 'sector': 'Technology',

In [9]:
from stock_functions import get_yf_stock_info, get_yf_stock_history, get_yf_stock_recommendations, tools

messages = [
    {"role": "system", "content": "너는 사용자를 도와주는 상담사야."},
]

user_input = "Apple 주식 정보는?"

messages.append({"role": "user", "content": user_input})

ai_response = get_ai_response(messages, tools=tools)
ai_message = ai_response.choices[0].message
print(ai_message) # GPT 응답 확인용

tool_calls = ai_message.tool_calls  # AI 응답에 포함된 tool_calls를 가져옵니다.
if tool_calls:  # tool_calls가 있는 경우
    for tool_call in tool_calls:
        tool_name = tool_call.function.name # 실행해야한다고 판단한 함수명 받기
        tool_call_id = tool_call.id         # tool_call 아이디 받기    
        arguments = json.loads(tool_call.function.arguments) # (1) 문자열을 딕셔너리로 변환    
        
        if tool_name == "get_yf_stock_info":  # 만약 tool_name이 "get_yf_stock_info"이라면
            messages.append({
                "role": "function",  # role을 "function"으로 설정
                "tool_call_id": tool_call_id,
                "name": tool_name,
                "content": get_yf_stock_info(arguments.get('ticker')),
            })
        elif tool_name == "get_yf_stock_history":  # 만약 tool_name이 "get_yf_stock_history"이라면
            messages.append({
                "role": "function",  # role을 "function"으로 설정
                "tool_call_id": tool_call_id,
                "name": tool_name,
                "content": get_yf_stock_history(arguments.get('ticker'), arguments.get('period')),
            })
        elif tool_name == "get_yf_stock_recommendations":  # 만약 tool_name이 "get_yf_stock_recommendations"이라면
            messages.append({
                "role": "function",  # role을 "function"으로 설정
                "tool_call_id": tool_call_id,
                "name": tool_name,
                "content": get_yf_stock_recommendations(arguments.get('ticker')),
            })
    messages.append({"role": "system", "content": "이제 주어진 결과를 바탕으로 답변할 차례다."})  # 함수 실행 완료 메시지 추가
    ai_response = get_ai_response(messages, tools=tools) # 다시 GPT 응답 받기
    ai_message = ai_response.choices[0].message

messages.append(ai_message)  # AI 응답을 대화 기록에 추가하기

print("AI\t: " + ai_message.content)

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_ThzOdWVgg2UiIfGLwFOL69hR', function=Function(arguments='{"ticker": "AAPL"}', name='get_yf_stock_info'), type='function'), ChatCompletionMessageFunctionToolCall(id='call_zdi4IsarxQc2WeBwCqQHOboD', function=Function(arguments='{"ticker": "AAPL", "period": "1d"}', name='get_yf_stock_history'), type='function'), ChatCompletionMessageFunctionToolCall(id='call_2qK5qb1hC7vtlfhiuyBFarPz', function=Function(arguments='{"ticker": "AAPL"}', name='get_yf_stock_recommendations'), type='function')])
{'address1': 'One Apple Park Way', 'city': 'Cupertino', 'state': 'CA', 'zip': '95014', 'country': 'United States', 'phone': '(408) 996-1010', 'website': 'https://www.apple.com', 'industry': 'Consumer Electronics', 'industryKey': 'consumer-electronics', 'industryDisp': 'Consumer Electronics', 'sector': 'Technology', 'sectorKey': 'tec